In [13]:
import os
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [14]:
!mkdir train_local
!unzip /content/gdrive/MyDrive/NFL_Helmet/nfl-health-and-safety-helmet-assignment.zip -d train_local/

流式输出内容被截断，只能显示最后 5000 行内容。
 extracting: train_local/nfl-health-and-safety-helmet-assignment/images/57830_000403_Sideline_frame0871.jpg  
  inflating: train_local/nfl-health-and-safety-helmet-assignment/images/57830_000734_Endzone_frame60.jpg  
 extracting: train_local/nfl-health-and-safety-helmet-assignment/images/57830_000734_Sideline_frame0520.jpg  
  inflating: train_local/nfl-health-and-safety-helmet-assignment/images/57830_000734_Sideline_frame89.jpg  
  inflating: train_local/nfl-health-and-safety-helmet-assignment/images/57830_000965_Endzone_frame192.jpg  
  inflating: train_local/nfl-health-and-safety-helmet-assignment/images/57830_000965_Sideline_frame3.jpg  
 extracting: train_local/nfl-health-and-safety-helmet-assignment/images/57830_001765_Sideline_frame0675.jpg  
  inflating: train_local/nfl-health-and-safety-helmet-assignment/images/57830_002030_Endzone_frame496.jpg  
  inflating: train_local/nfl-health-and-safety-helmet-assignment/images/57830_002030_Sideline_frame191.jp

In [15]:
import random
import numpy as np
from pathlib import Path
import datetime
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
import cv2
import json
import matplotlib.pyplot as plt
from IPython.core.display import Video, display
import subprocess
import gc
import shutil

In [17]:
os.listdir("./train_local")

['nfl-health-and-safety-helmet-assignment']

# prepare data as COCO format (standard format of obeject detection)
# reference https://www.kaggle.com/eneszvo/mmdet-cascadercnn-helmet-detection-for-beginners

In [16]:
import pandas as pd

# Load image level csv file
extra_df = pd.read_csv('./train_local/image_labels.csv')
print('Number of ground truth bounding boxes: ', len(extra_df))

# Number of unique labels
label_to_id = {label: i for i, label in enumerate(extra_df.label.unique())}
print('Unique labels: ', label_to_id)

FileNotFoundError: ignored

In [ ]:
extra_df.head()

In [ ]:
def create_ann_file(df, category_id):
    
    now = datetime.datetime.now()

    data = dict(
        info=dict(
            description='NFL-Helmet-Assignment',
            url=None,
            version=None,
            year=now.year,
            contributor=None,
            date_created=now.strftime('%Y-%m-%d %H:%M:%S.%f'),
        ),
        licenses=[dict(
            url=None,
            id=0,
            name=None,
        )],
        images=[
            # license, url, file_name, height, width, date_captured, id
        ],
        type='instances',
        annotations=[
            # segmentation, area, iscrowd, image_id, bbox, category_id, id
        ],
        categories=[
            # supercategory, id, name
        ],
    )
    
    class_name_to_id = {}
    labels =  ["__ignore__",
                'Helmet',
              'Helmet-Blurred', 
              'Helmet-Difficult', 
              'Helmet-Sideline',
              'Helmet-Partial']

    for i, each_label in enumerate(labels):
        class_id = i - 1  # starts with -1
        class_name = each_label
        if class_id == -1:
            assert class_name == '__ignore__'
            continue
        class_name_to_id[class_name] = class_id
        data['categories'].append(dict(
            supercategory=None,
            id=class_id,
            name=class_name,
        ))
    
    box_id = 0
    for i, image in tqdm(enumerate(os.listdir(TRAIN_PATH))):

        img = cv2.imread(TRAIN_PATH+'/'+image)
        height, width, _ = img.shape

        data['images'].append({
            'license':0, 
            'url': None,
            'file_name': image,
            'height': height,
            'width': width,
            'date_camputured': None,
            'id': i
        })

        df_temp = df[df.image == image]
        for index, row in df_temp.iterrows():

            area = round(row.width*row.height, 1)
            bbox =[row.left, row.top, row.width, row.height]

            data['annotations'].append({
                'id': box_id,
                'image_id': i,
                'category_id': category_id[row.label],
                'area': area,
                'bbox':bbox,
                'iscrowd':0
            })
            box_id+=1
    
    return data

In [ ]:
from sklearn.model_selection import train_test_split

TRAIN_PATH = './train_local/images'
extra_df = pd.read_csv('./train_local/image_labels.csv')

category_id = {'Helmet':0, 'Helmet-Blurred':1,
               'Helmet-Difficult':2, 'Helmet-Sideline':3,
               'Helmet-Partial':4}

df_train, df_val = train_test_split(extra_df, test_size=0.2, random_state=36)

ann_file_train = create_ann_file(df_train, category_id)
ann_file_val = create_ann_file(df_val, category_id)

In [ ]:
print('train:',df_train.shape,'val:',df_val.shape)

In [ ]:
#save as json to gdrive
with open('/content/gdrive/MyDrive/NFL_Helmet/ann_file_train.json', 'w') as f:
    json.dump(ann_file_train, f, indent=4)
        
with open('/content/gdrive/MyDrive/NFL_Helmet/ann_file_val.json', 'w') as f:
    json.dump(ann_file_val, f, indent=4)